In [0]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive')#change dir

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
from keras.preprocessing import image #// test checl
import os
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2

Using TensorFlow backend.


In [0]:
df_train = pd.read_excel('/content/drive/My Drive/Labels_1.xlsx')
df_train.head()

,ImageName,Level
0,scen10_cam200095.jpg,1
1,scen10_cam200096.jpg,1
2,scen10_cam200097.jpg,1
3,scen10_cam200098.jpg,1
4,scen10_cam200099.jpg,1


In [0]:
targets_series = pd.Series(df_train['Level'])
one_hot = pd.get_dummies(targets_series, sparse = True)

In [0]:
one_hot_labels = np.asarray(one_hot)

In [0]:
im_size1 = 224 #deafult is 224 /given is 256 
im_size2 = 224 #deafult is 224 /given is 256 

In [0]:
x_train = []
y_train = []
x_test = []

In [0]:
# i = 0 
# #for folder in range(9):
# for f, breed in tqdm(df_train.values[:6786]):
#   if type(cv2.imread('Train/{}'.format(f)))==type(None):
#     continue 
#   else: 
#     img = cv2.imread('Train/{}'.format(f)) 
#     label = one_hot_labels[i] 
#     x_train.append(cv2.resize(img, (im_size1, im_size2))) 
#     y_train.append(label) 
#     i += 1 
# np.save('x_train2',x_train) 
# np.save('y_train2',y_train) 
# print('Done')

In [0]:
x_train = np.load('x_train2.npy')
y_train = np.load('y_train2.npy')

In [0]:
y_train_raw = np.array(y_train, np.uint8)
x_train_raw = np.array(x_train, np.float32) / 255.

In [0]:
print(x_train_raw.shape)
print(y_train_raw.shape)

(6785, 224, 224, 3)
(6785, 2)


In [0]:
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=0.1, random_state=1)

In [0]:
num_class = y_train_raw.shape[1]

In [0]:
from keras.applications.resnet50 import ResNet50

In [0]:
base_model = ResNet50(weights = None, include_top=False, input_shape=(im_size1, im_size2, 3))

# Add a new top layer
x = base_model.output
x = Flatten()(x)
x = Dropout(0.2)(x)
x = Dense(32, activation='relu')(x)
x = Dense(16, activation='relu')(x)
predictions = Dense(num_class, activation='softmax')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# First: train only the top layers (which were randomly initialized)
#for layer in base_model.layers:
#    layer.trainable = False

model.compile(loss='categorical_crossentropy', 
              optimizer='rmsprop', 
              metrics=['accuracy'])

callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', verbose=1)]
model.summary()

In [0]:
# base_model = ResNet50(weights = None, include_top=False, input_shape=(im_size1, im_size2, 3))

# # Add a new top layer
# x = base_model.output
# x = Flatten()(x)
# x = Dropout(0.2)(x)
# x = Dense(32, activation='relu')(x)
# x = Dense(16, activation='relu')(x)
# predictions = Dense(num_class, activation='softmax')(x)

# # This is the model we will train
# model = Model(inputs=base_model.input, outputs=predictions)

# # First: train only the top layers (which were randomly initialized)
# #for layer in base_model.layers:
# #    layer.trainable = False

# model.compile(loss='categorical_crossentropy', 
#               optimizer='Adagrad', 
#               metrics=['accuracy'])

# callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', verbose=1)]
# model.summary()

#125ms/step - loss: 9.0833 - acc: 0.4365 - val_loss: 9.1866 - val_acc: 0.4300

In [0]:
base_model = ResNet50(weights = 'imagenet', include_top=False, input_shape=(im_size1, im_size2, 3))

# Add a new top layer
x = base_model.output
x = Flatten()(x)
x = Dropout(0.4)(x)
# x = Dense(1024, activation='relu')(x)
# x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dense(32, activation='relu')(x)
x = Dense(16, activation='relu')(x)
predictions = Dense(num_class, activation='softmax')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# First: train only the top layers (which were randomly initialized)
#for layer in base_model.layers:
#    layer.trainable = False
from keras.optimizers import RMSprop
#rmsprop = RMSprop(lr=0.0001)
model.compile(loss='categorical_crossentropy', 
              optimizer='rmsprop', #Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)124ms/step - loss: 9.0833 - acc: 0.4365 - val_loss: 9.1866 - val_acc: 0.4300
              metrics=['accuracy'])

callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', verbose=1)]
model.summary()

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 8s 0us/step
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________

In [0]:
model.fit(X_train, Y_train,  epochs=40, validation_data=(X_valid, Y_valid), verbose=1)

Train on 6106 samples, validate on 679 samples
Epoch 1/40
6106/6106 [==============================] - 87s 14ms/step - loss: 0.8514 - accuracy: 0.7204 - val_loss: 29656.3308 - val_accuracy: 0.5700
Epoch 2/40
6106/6106 [==============================] - 68s 11ms/step - loss: 0.4211 - accuracy: 0.8387 - val_loss: 0.7437 - val_accuracy: 0.4300
Epoch 3/40
6106/6106 [==============================] - 68s 11ms/step - loss: 0.3022 - accuracy: 0.8927 - val_loss: 0.7407 - val_accuracy: 0.4300
Epoch 4/40
6106/6106 [==============================] - 68s 11ms/step - loss: 0.2441 - accuracy: 0.9063 - val_loss: 0.7250 - val_accuracy: 0.4300
Epoch 5/40
6106/6106 [==============================] - 68s 11ms/step - loss: 0.2075 - accuracy: 0.9232 - val_loss: 0.4834 - val_accuracy: 0.7541
Epoch 6/40
6106/6106 [==============================] - 68s 11ms/step - loss: 0.1797 - accuracy: 0.9335 - val_loss: 0.7809 - val_accuracy: 0.8557
Epoch 7/40
6106/6106 [==============================] - 68s 11ms/step - l

In [0]:
# import matplotlib.pyplot as plt
# import numpy as np

In [0]:
# epochs = 1

In [0]:
#model.fit(X_train, Y_train,  epochs=1, validation_data=(X_valid, Y_valid), verbose=1)
#model.fit(X_train, Y_train, epochs=epochs, steps_per_epoch=150,validation_steps=100, validation_data=(X_valid, Y_valid), verbose=1)
#model.fit(X_train, Y_train, epochs=epochs, validation_data=(X_valid, Y_valid), verbose=1).history
#history=model.fit(X_train, Y_train, epochs=epochs, validation_data=(X_valid, Y_valid), verbose=1).history


In [0]:
N = epochs
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), history["loss"], label="train_loss")
plt.plot(np.arange(0, N), history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), history["acc"], label="test_acc")
plt.plot(np.arange(0, N), history["val_acc"], label="val_acc")
plt.title("Optimized ResNet-50 Results ")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("result.png")

NameError: ignored

In [0]:
# from keras.utils.vis_utils import plot_model

In [0]:
# plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
